# Object Storage에서 Parquet 파일 읽기 예제


## 1) 필요한 라이브러리 설치


In [ ]:
pip install boto3 pandas deltalake pyarrow==15.0.2

## 2) AWS S3 클라이언트 생성

- `S3_ACCESS_KEY`, `S3_SECRET_KEY` 에 실제 환경 변수 값으로 수정

In [ ]:
import boto3

ACCESS_KEY = "{S3_ACCESS_KEY_ID}"
SECRET_KEY = "{S3_SECRET_ACCESS_KEY}"

s3 = boto3.client(
    service_name="s3",
    region_name="kr-central-2",
    endpoint_url="https://objectstorage.kr-central-2.kakaocloud.com",
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY
)